<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Time-correction" data-toc-modified-id="Time-correction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Time correction</a></span><ul class="toc-item"><li><span><a href="#Get-accquisition-time" data-toc-modified-id="Get-accquisition-time-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get accquisition time</a></span></li><li><span><a href="#Get-the-label-file" data-toc-modified-id="Get-the-label-file-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Get the label file</a></span></li><li><span><a href="#Reset-the-time" data-toc-modified-id="Reset-the-time-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Reset the time</a></span></li><li><span><a href="#Write-back-to-the-label-file" data-toc-modified-id="Write-back-to-the-label-file-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Write back to the label file</a></span></li></ul></li></ul></div>

## Time correction
When you forgot to change the acquisition time, and you had already labeled some data, you can use this script to change the acquisition time of you label file.

In [124]:
import datetime
import pandas as pd

Some self defined functions

In [155]:
def transfer_time(date_time, seconds, date_time_format='%d:%H:%M:%S'):
    """
    Add seconds to the date time and transfer to the target format
    
    Parameters
    ----------
    date_time : datetime object
        The date time we want to start with, here is the reset acquisition time
    seconds : int
        Seconds going to add to the date_time
    date_time_format : str
        Final format of date_time. Defaults is '%d:%M:%H:%S'
        
    Returns
    -------
    target_time : str
        Final date time in string format
        
    Examples
    --------
    Add seconds to the datetime
    
    >>> import datetime
    >>> original_time = datetime.datetime(2024, 1, 30, 10, 50, 0)
    >>> seconds = 40
    >>> format_ = '%d-%M:%H:%S'
    >>> transfer_time(original_time, seconds, format_)
    '30-10:50:40'
    """
    
    temp_time = date_time + datetime.timedelta(seconds=seconds)
    return temp_time.strftime(format=date_time_format)

def label2df(labels, correct_time):
    """
    Transfer the original label to dataframe and correct the date time

    Parameters
    ----------
    labels : List
        Marker, stat-end or sleep stage labels
    correct_time : datetime object
        Time for correction

    Returns
    -------
    result_df : dataframe had corrected time
    """

    # Separate the data by `, ` to get a dataframe of sleep stage labels
    df = pd.DataFrame(data=labels, columns=['string'])
    df = df['string'].str.split(', ', expand=True)
    df.columns = ['start_time', 'start_time_sec', 'start_code',
                  'end_time', 'end_time_sec', 'end_code',
                  'state_code', 'state']

    df['start_time'] = df['start_time_sec'].apply(
        lambda x: transfer_time(correct_time, int(x)))

    df['end_time'] = df['end_time_sec'].apply(
        lambda x: transfer_time(correct_time, int(x)))

    return df

def df2str(df, separator=', '):
    """
    Transfer the dataframe to a string object
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe to transfer
    separator : str
        Separator used to split the dataframe columns. Defaults is `, `
        
    Returns
    -------
    result_str : str
    """
    
    str_lst = []
    for row in df.iterrows():
        str_lst.append(separator.join(list(row[1])))
    return '\n'.join(str_lst)

### Get accquisition time
Reset a time for the acquisition time

With `datetime` module, we can transfer a string object into a datetime object, so that we can get the year, month ... parts respectively.

In [156]:
reset_acquisition_time_str = '2023-10-01 19:00:00'
reset_acquisition_time = datetime.datetime.strptime(reset_acquisition_time_str, 
                                           '%Y-%m-%d %H:%M:%S')
print(f'Year: {reset_acquisition_time.year}, \n'
      f'Month: {reset_acquisition_time.month}, \n'
      f'Day: {reset_acquisition_time.day}, \n'
      f'Hour: {reset_acquisition_time.hour}, \n'
      f'Minute: {reset_acquisition_time.minute}, \n'
      f'Second: {reset_acquisition_time.second}')

Year: 2023, 
Month: 10, 
Day: 1, 
Hour: 19, 
Minute: 0, 
Second: 0


And the `datetime.timedelta` function allows us to add some seonds to arrive the time we want.

In [127]:
# Run to the doc for detail
?datetime.timedelta

In [151]:
# Here we add 86401 seconds (which is one day and a second) to the original time
arrived_time = reset_acquisition_time + datetime.timedelta(seconds=86401)
print(arrived_time)

2023-10-02 19:00:01


With `datetime.datetime.strftime` we can transfer the date time object to the target format

In [145]:
arrived_time.strftime(format='%d:%M:%H:%S')

'02:00:19:01'

### Get the label file

In [165]:
# Read out the label file and split by `\n` to get each line
label_file_path = './data/test_label.txt'
label_file = open(label_file_path, 'r').read().split('\n')
print('\n'.join(label_file[:5]))

# Get the `==========Marker==========` part
marker = label_file[label_file.index('==========Marker==========')+1: 
                    label_file.index('==========Start-End==========')]

# Get the `==========Start-End==========` part
start_end = label_file[label_file.index('==========Start-End==========')+1: 
                       label_file.index('==========Sleep stage==========')]

# Only get the `==========Sleep stage==========` part
sleep_stage = label_file[label_file.index('==========Sleep stage==========')+1: ]
print('\n'.join(sleep_stage[:5]))

READ ONLY! DO NOT EDIT!
4-INIT 3-Wake 2-REM 1-NREM
Save time: 2024-01-30 12:57:08
Acquisition time: 2024-01-28 12:49:00
Sampling rate: 305
28:12:49:00, 0, 1, 28:13:00:27, 687, 0, 3, Wake
28:13:00:28, 688, 1, 28:13:00:46, 706, 0, 1, NREM
28:13:00:47, 707, 1, 28:13:00:55, 715, 0, 3, Wake
28:13:00:56, 716, 1, 28:13:03:23, 863, 0, 1, NREM
28:13:03:24, 864, 1, 28:13:03:29, 869, 0, 3, Wake


### Reset the time
With the `timedelta` function and the `absolute second` of each row, we can easily transfer the time in the dataframe


In [160]:
marker_str = ''
if len(marker) > 0:
    marker_df = label2df(labels=marker, correct_time=reset_acquisition_time)
    marker_str = '\n' + df2str(marker_df)

start_end_str = ''
if len(start_end) > 0:
    start_end_df = label2df(labels=start_end, correct_time=reset_acquisition_time)
    start_end_str = '\n' + df2str(start_end_df)

sleep_stage_df = label2df(labels=sleep_stage, correct_time=reset_acquisition_time)
sleep_stage_str = df2str(sleep_stage_df)

sleep_stage_df.head()

,start_time,start_time_sec,start_code,end_time,end_time_sec,end_code,state_code,state
0,01:19:00:00,0,1,01:19:11:27,687,0,3,Wake
1,01:19:11:28,688,1,01:19:11:46,706,0,1,NREM
2,01:19:11:47,707,1,01:19:11:55,715,0,3,Wake
3,01:19:11:56,716,1,01:19:14:23,863,0,1,NREM
4,01:19:14:24,864,1,01:19:14:29,869,0,3,Wake


### Write back to the label file
Transfer the dataframe into string format and change the Acquisition time in the head information, and write back to the label file.

In [161]:
# Get the current time in target format
datetime.datetime.now().strftime(format='%Y-%m-%d %H:%M:%S')

'2024-01-30 13:00:13'

In [162]:
# Construct the string for writing
save_time = datetime.datetime.now().strftime(format='%Y-%m-%d %H:%M:%S')
write_str = str(f'READ ONLY! DO NOT EDIT!\n'
                f'4-INIT 3-Wake 2-REM 1-NREM\n'
                f'Save time: {save_time}\n'
                f"Acquisition time: {reset_acquisition_time_str}\n"
                f'{label_file[4]}\n'
                + '==========Marker==========' + marker_str
                + '\n==========Start-End==========' + start_end_str
                + f'\n==========Sleep stage==========\n'
                  f'{sleep_stage_str}'
                )

In [163]:
print(write_str)

READ ONLY! DO NOT EDIT!
4-INIT 3-Wake 2-REM 1-NREM
Save time: 2024-01-30 13:00:17
Acquisition time: 2023-10-01 19:00:00
Sampling rate: 305
==========Marker==========
==========Start-End==========
==========Sleep stage==========
01:19:00:00, 0, 1, 01:19:11:27, 687, 0, 3, Wake
01:19:11:28, 688, 1, 01:19:11:46, 706, 0, 1, NREM
01:19:11:47, 707, 1, 01:19:11:55, 715, 0, 3, Wake
01:19:11:56, 716, 1, 01:19:14:23, 863, 0, 1, NREM
01:19:14:24, 864, 1, 01:19:14:29, 869, 0, 3, Wake
01:19:14:30, 870, 1, 01:19:15:33, 933, 0, 1, NREM
01:19:15:34, 934, 1, 01:19:15:39, 939, 0, 3, Wake
01:19:15:40, 940, 1, 01:19:18:14, 1094, 0, 1, NREM
01:19:18:15, 1095, 1, 01:19:18:56, 1136, 0, 2, REM
01:19:18:57, 1137, 1, 01:19:19:13, 1153, 0, 3, Wake
01:19:19:14, 1154, 1, 01:19:23:40, 1420, 0, 1, NREM
01:19:23:41, 1421, 1, 01:19:23:53, 1433, 0, 3, Wake
01:19:23:54, 1434, 1, 01:19:24:36, 1476, 0, 1, NREM
01:19:24:37, 1477, 1, 01:19:24:43, 1483, 0, 3, Wake
01:19:24:44, 1484, 1, 01:19:25:56, 1556, 0, 1, NREM
01:19:25:5

In [140]:
# Write to the label file
with open (label_file_path, 'w') as f:
    f.write(write_str)
